In [1]:
import pandas as pd
import numpy as np
import string
import resource
import os, re
import array


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
dataset_dw = pd.read_csv("/content/gdrive/MyDrive/dataset_large_dw.csv")
dataset_wikipedia_test = pd.read_csv("/content/gdrive/MyDrive/dataset_wikipedia.csv")


Mounted at /content/gdrive


In [3]:
# big_dataset = pd.concat([dataset_wikipedia, dataset_dw], ignore_index=True)
big_dataset = dataset_dw

In [4]:
def preprocess(big_dataset):  
  big_dataset["ID"] = big_dataset["ID"].map(lambda name: name.lower())
  special = re.compile(r'\W')
  single = re.compile(r'\s+', flags=re.I)
  number = re.compile(r"[-+]?[0-9]+")
  pnumber = re.compile(r"[-+]?[\u06F0-\u06F90-9]+")
  anumber = re.compile(r"[-+]?[\u0660-\u0669]+")
  url = re.compile(r"https?://\S+|www\.\S+")
  html = re.compile(r"<.*?>")
  emoji_pattern = re.compile(
      "["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      "]+",
      flags=re.UNICODE,
  )

  big_dataset["ID"] = big_dataset["ID"].map(lambda x: url.sub(r" ",x))
  big_dataset["ID"] = big_dataset["ID"].map(lambda x: html.sub(r" ",x))
  big_dataset["ID"] = big_dataset["ID"].map(lambda x: emoji_pattern.sub(r" ",x))
  big_dataset["ID"] = big_dataset["ID"].map(lambda x: number.sub(r" ",x))
  big_dataset["ID"] = big_dataset["ID"].map(lambda x: pnumber.sub(r" ",x))
  big_dataset["ID"] = big_dataset["ID"].map(lambda x: anumber.sub(r" ",x))
  big_dataset["ID"] = big_dataset["ID"].map(lambda x: x.translate(str.maketrans(" ", " ", string.punctuation)))
  big_dataset["ID"] = big_dataset["ID"].map(lambda x: special.sub(r" ",x))
  big_dataset["ID"] = big_dataset["ID"].map(lambda x: single.sub(r" ", x))

  return big_dataset


In [5]:
big_dataset = preprocess(big_dataset)
dataset_wikipedia_test = preprocess(dataset_wikipedia_test)
df = big_dataset.copy()
df.Category = df.Category.astype('category')
nb_classes = df.Category.cat.codes.max() + 1
df["CategoryCodes"] = df.Category.cat.codes

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.ID, df.Category, test_size=0.1, random_state=42)

In [7]:
import math

def vec_fit_transform(X_train):
  DF = {}
  for index in X_train.index:
      for w in X_train[index].split():
          try:
              DF[w].add(index)
          except:
              DF[w] = {index}

  N = len(X_train)

  IDF = DF
  for i in DF:
    # IDF[i] = math.log((N + 1) / float((len(IDF[i]) + 1))) + 1
    IDF[i] = math.log((N) / float((len(IDF[i]) + 1)))


  TF_IDF = {}
  VECTORIZE = dict()
  for index in X_train.index:
      tokens = X_train[index].split()
      words_count = len(tokens)
      weight_list = []
      for token in tokens:
          TF = list(tokens).count(token) / float(words_count)
          RES = TF*IDF[token]
          TF_IDF[index, token] = RES
          weight_list.append(RES)
      try:
          VECTORIZE[index].append(weight_list)
      except:
          VECTORIZE[index] = weight_list

  index_mapper = {}
  index_mapper_cnt = 0
  for i in list(X_train.index):
    index_mapper[i] = index_mapper_cnt
    index_mapper_cnt += 1

  print(index_mapper)

  VOCAB_LIST = [x for x in DF]
  D = np.zeros((len(X_train), len(VOCAB_LIST)))
  for i in TF_IDF:
      # print(i)
      ind = VOCAB_LIST.index(i[1])
      D[index_mapper[i[0]]][ind] = TF_IDF[i]
  print(D[0])

  DATAFRAME = pd.DataFrame(D)

  print(DATAFRAME)
  # DATAFRAME.apply(np.linalg.norm, axis=1)
  # print(DATAFRAME)

  return DATAFRAME, IDF, DF, len(DF)

In [8]:
import math

def vec_transform(X_test, IDF, DF):

  TF_IDF = {}
  VECTORIZE = dict()
  for index in X_test.index:
      tokens = X_test[index].split()
      words_count = len(tokens)
      weight_list = []
      for token in tokens:
          TF = list(tokens).count(token) / float(words_count)
          try:
            RES = TF*IDF[token]
          except:
            RES = 0
          TF_IDF[index, token] = RES
          weight_list.append(RES)
      try:
          VECTORIZE[index].append(weight_list)
      except:
          VECTORIZE[index] = weight_list

  index_mapper = {}
  index_mapper_cnt = 0
  for i in list(X_test.index):
    index_mapper[i] = index_mapper_cnt
    index_mapper_cnt += 1

  print(index_mapper)

  VOCAB_LIST = [x for x in DF]
  D = np.zeros((len(X_test), len(VOCAB_LIST)))
  for i in TF_IDF:
      # print(i)
      try:
        ind = VOCAB_LIST.index(i[1])
        D[index_mapper[i[0]]][ind] = TF_IDF[i]
      except:
        pass
  print(D[0])

  DATAFRAME = pd.DataFrame(D)

  print(DATAFRAME)
  # DATAFRAME.apply(np.linalg.norm, axis=1)
  # print(DATAFRAME)

  return DATAFRAME

In [9]:
vectorized_X_train, IDF, DF, vocab_size = vec_fit_transform(X_train)
# vocab_size = 6

{4643: 0, 381: 1, 1297: 2, 3977: 3, 422: 4, 2405: 5, 4993: 6, 1193: 7, 1873: 8, 124: 9, 102: 10, 4503: 11, 4279: 12, 5746: 13, 994: 14, 1894: 15, 4303: 16, 2333: 17, 3347: 18, 135: 19, 811: 20, 2018: 21, 4794: 22, 2451: 23, 5352: 24, 1215: 25, 1272: 26, 4001: 27, 3381: 28, 2350: 29, 1820: 30, 2029: 31, 3567: 32, 1049: 33, 4634: 34, 1881: 35, 1480: 36, 4883: 37, 144: 38, 380: 39, 3560: 40, 915: 41, 3254: 42, 121: 43, 611: 44, 3307: 45, 5737: 46, 418: 47, 5206: 48, 1281: 49, 1223: 50, 4187: 51, 783: 52, 5683: 53, 3352: 54, 1543: 55, 2093: 56, 248: 57, 996: 58, 3918: 59, 447: 60, 2450: 61, 879: 62, 1172: 63, 2746: 64, 3933: 65, 4742: 66, 881: 67, 2981: 68, 3435: 69, 3269: 70, 2464: 71, 414: 72, 1703: 73, 5430: 74, 654: 75, 1101: 76, 5741: 77, 3764: 78, 3966: 79, 2086: 80, 787: 81, 5198: 82, 589: 83, 4368: 84, 4563: 85, 5847: 86, 3699: 87, 712: 88, 4314: 89, 2741: 90, 3628: 91, 5845: 92, 1302: 93, 3315: 94, 208: 95, 5337: 96, 4356: 97, 425: 98, 1335: 99, 5566: 100, 5196: 101, 4727: 102, 36

In [10]:
vectorized_X_test = vec_transform(X_test, IDF, DF)

{5579: 0, 3443: 1, 2351: 2, 4456: 3, 2742: 4, 705: 5, 1618: 6, 4366: 7, 3210: 8, 296: 9, 2815: 10, 1295: 11, 5263: 12, 2053: 13, 3524: 14, 1544: 15, 808: 16, 5494: 17, 794: 18, 3862: 19, 1097: 20, 4707: 21, 3692: 22, 4955: 23, 319: 24, 1786: 25, 465: 26, 1870: 27, 5485: 28, 2767: 29, 1777: 30, 4063: 31, 4771: 32, 3678: 33, 1468: 34, 1039: 35, 3716: 36, 1580: 37, 5601: 38, 2536: 39, 233: 40, 805: 41, 4355: 42, 2357: 43, 4678: 44, 1212: 45, 177: 46, 681: 47, 5246: 48, 23: 49, 3745: 50, 420: 51, 4269: 52, 1310: 53, 960: 54, 676: 55, 2406: 56, 1755: 57, 1747: 58, 755: 59, 401: 60, 84: 61, 5604: 62, 1350: 63, 2447: 64, 4414: 65, 4985: 66, 1244: 67, 2422: 68, 3248: 69, 3240: 70, 1321: 71, 4605: 72, 2780: 73, 1962: 74, 3050: 75, 4915: 76, 4842: 77, 2945: 78, 5740: 79, 4746: 80, 2515: 81, 3858: 82, 3358: 83, 1109: 84, 4852: 85, 746: 86, 2251: 87, 3833: 88, 1532: 89, 4211: 90, 598: 91, 2841: 92, 4200: 93, 4618: 94, 4956: 95, 3370: 96, 4189: 97, 5538: 98, 4639: 99, 157: 100, 2095: 101, 2321: 102

In [11]:
dataset_wikipedia_test.ID

0       molecular imaging is broadly defined as the vi...
1       setor de indústria e abastecimento is an admin...
2        pajero group urdu and punjabi پجارو گروپ is a...
3       the gau lower silesia german gau niederschlesi...
4       a young person called traffic cadet german ver...
                              ...                        
2994    اسپانیا له جغرافیایي پلوه نیمه ګرمه او معتدله ...
2995                                 په رياضي کې کولاي سي
2996    پوردوممه په لومړیو وختونو کښي غوښتل چي انجینر ...
2997                                 کينډۍد هرمزگان ولايت
2998                            کينډۍد خراسان شمالي ولايت
Name: ID, Length: 2999, dtype: object

In [12]:
wiki_vectorized_X_test = vec_transform(dataset_wikipedia_test.ID, IDF, DF)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121,

In [13]:
dataset_wikipedia_test.Category

0       English
1       English
2       English
3       English
4       English
         ...   
2994     Pashto
2995     Pashto
2996     Pashto
2997     Pashto
2998     Pashto
Name: Category, Length: 2999, dtype: object

In [14]:
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV



# classifier = RandomForestClassifier(n_estimators=200, random_state=0)
# classifier = SGDClassifier(loss="modified_huber", max_iter=10000, alpha=0.00001)
classifier = LinearSVC()
# classifier = LogisticRegression(max_iter=1000)


classifier.fit(vectorized_X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [15]:
############################################################################# splited test set

y_pred = classifier.predict(vectorized_X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[113   0   0   0   1   0]
 [  0  98   0   0   0   0]
 [  0   1 117   0   0   0]
 [  0   0   0  94   0   0]
 [  0   0   0   0  93   0]
 [  0   0   0   0   0  75]]
              precision    recall  f1-score   support

      Arabic       1.00      0.99      1.00       114
     English       0.99      1.00      0.99        98
      French       1.00      0.99      1.00       118
      German       1.00      1.00      1.00        94
      Pashto       0.99      1.00      0.99        93
     Persian       1.00      1.00      1.00        75

    accuracy                           1.00       592
   macro avg       1.00      1.00      1.00       592
weighted avg       1.00      1.00      1.00       592

0.9966216216216216


In [16]:
############################################################################# wikipedia test set

y_pred = classifier.predict(wiki_vectorized_X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(dataset_wikipedia_test.Category,y_pred))
print(classification_report(dataset_wikipedia_test.Category,y_pred))
print(accuracy_score(dataset_wikipedia_test.Category, y_pred))

[[487   2   1   0   8   2]
 [  0 497   1   0   2   0]
 [  0   2 493   0   5   0]
 [  0   1   0 496   3   0]
 [  2   8   1   0 485   3]
 [  0   7   0   0  13 480]]
              precision    recall  f1-score   support

      Arabic       1.00      0.97      0.98       500
     English       0.96      0.99      0.98       500
      French       0.99      0.99      0.99       500
      German       1.00      0.99      1.00       500
      Pashto       0.94      0.97      0.96       499
     Persian       0.99      0.96      0.97       500

    accuracy                           0.98      2999
   macro avg       0.98      0.98      0.98      2999
weighted avg       0.98      0.98      0.98      2999

0.9796598866288763


In [17]:
############################################################################# kaggle test set

df_test = pd.read_csv('/content/gdrive/MyDrive/kaggle.csv')
print(df_test)
df_test_values = df_test["Id"].copy()
df_test.columns = ['ID', 'Category']
df_test = preprocess(df_test)
df_test.columns = ['Id', 'Category']

print(df_test)
X = vec_transform(df_test["Id"], IDF, DF)
Y_final = classifier.predict(X)

df_test_res = df_test.copy()
df_test_res["Category"] = Y_final
df_test_res["Id"] = df_test_values
print(df_test_res)

# df_test_res = df_test_res.rename(columns = {"Id":"Category"}) 
df_test_res.to_csv("Final.csv", index=False)



                                                   Id Category
0   واتهمت وزارة العدل الأمركية جون مكافي بالتهرب ...   Arabic
1   Eine propositionale Einstellung (englisch prop...   German
2   Rule-based machine learning (RBML) is a term i...  English
3   In a genetic algorithm, a population of candid...  English
4   Its fields can be divided into theoretical and...  English
..                                                ...      ...
95  اللُّغَة العَرَبِيّة هي أكثر اللغات السامية تح...  Persian
96  ريتشارد ماثيو ستالمان (بالإنجليزية: Richard Ma...  Persian
97  Bob Dylan (born Robert Allen Zimmerman; May 24...  Persian
98  بعضی از ثروتمندترین و معروفترین افراد دنیا همی...  Persian
99  Das Land Baden-Württemberg verteilt Impfdosen ...  Persian

[100 rows x 2 columns]
                                                   Id Category
0   واتهمت وزارة العدل الأمركية جون مكافي بالتهرب ...   Arabic
1   eine propositionale einstellung englisch propo...   German
2   rulebased machine learning 

In [18]:
def sgd_optimize(X_train, y_train, vocab_size, lr_rate, lr_rate_variation, n_epochs, power_epoch):

    lang_mapper = {"English":0, "German":1, "French":2, "Persian":3, "Arabic":4, "Pashto":5}

    try:
        with open('weights.npy', 'rb') as f:
          w_coeff = np.load(f)
          b_coeff = np.load(f)
          print("Weights File is available!")
    except:
        w_coeff=np.zeros((vocab_size,1)) #Randomly initalizing weights
        b_coeff=np.zeros((1,1))  #Randomly picking up intercept value.
        print("Weights File is not available!")

    
    for epoch in range(1,n_epochs+1):
        sum_errors = 0 #Sum of squared loss.
        N = X_train.shape[0] #The variable N in the SGD equation.


        for i in range(N):
            batch_size = np.random.randint(0,N)  # random batch size for every iteration i.e k batch_size
            


            X_i = X_train[batch_size,:].reshape(1,X_train.shape[1])
            # print(np.array([batch_size]))

            y_i = np.array(lang_mapper[y_train[batch_size]]).reshape(1,1)
            # y_i = y_train[batch_size].reshape(1,1)


            y_pred = np.dot(X_i,w_coeff) + b_coeff                  #y_curr = WT.X + B
            # print(y_pred)
            # print(y_i)
            loss = y_pred - y_i                                     #w_grad = (-2/N)*(X)*(y-(WT.X+B)) = (-2/N)*(X) * loss 
            sum_errors += loss**2                                   #b_grad = (-2/N)*(y-(WT.X+B)) = (-2/N) * loss

            w_grad = X_i.T.dot((y_pred - y_i))
            b_grad = (y_pred - y_i)

            w_coeff = w_coeff -(2/N)*lr_rate*(w_grad)
            b_coeff = b_coeff - (2/N)*lr_rate*(b_grad)

        print("Epoch: %d, Loss: %.3f" %(epoch, sum_errors/N))

        if(lr_rate_variation=='invscaling'): #Implementing learning_rate 'invscaling' similar to that present in SGD Regressor.
            lr_rate = lr_rate / pow(epoch, power_epoch)
        else:
            pass

    return w_coeff, b_coeff

#This function is used to predict the class values given a test data.
def predict(X_test, w_coeff, b_coeff):
    lang_mapper = {0:"English", 1:"German", 2:"French", 3:"Persian", 4:"Arabic", 5:"Pashto"}
    X_test=np.array(X_test)
    y_pred =[]
    for i in range(0,len(X_test)):
        # print(w_coeff)
        # print(X_test[i].reshape(1, -1))
        # print(b_coeff)
        y=np.asscalar(np.dot(w_coeff,X_test[i]) + b_coeff) #Convert an array of size 1 to its scalar equivalent.
        y_pred.append(y)
        # y_pred.append(lang_mapper[int(round(y))])
    return np.array(y_pred)

In [ ]:
w_coeff_optimal, b_coeff_optimal = sgd_optimize(vectorized_X_train.to_numpy(), y_train.to_numpy(), vocab_size,lr_rate=1, lr_rate_variation='constant', n_epochs=250, power_epoch=None)

Weights File is available!
Epoch: 1, Loss: 2.773
Epoch: 2, Loss: 2.666
Epoch: 3, Loss: 2.620
Epoch: 4, Loss: 2.539
Epoch: 5, Loss: 2.402
Epoch: 6, Loss: 2.328
Epoch: 7, Loss: 2.267
Epoch: 8, Loss: 2.183
Epoch: 9, Loss: 2.148
Epoch: 10, Loss: 2.073
Epoch: 11, Loss: 2.024
Epoch: 12, Loss: 1.933
Epoch: 13, Loss: 1.876
Epoch: 14, Loss: 1.793
Epoch: 15, Loss: 1.776
Epoch: 16, Loss: 1.693
Epoch: 17, Loss: 1.649
Epoch: 18, Loss: 1.553
Epoch: 19, Loss: 1.539
Epoch: 20, Loss: 1.498
Epoch: 21, Loss: 1.463
Epoch: 22, Loss: 1.428
Epoch: 23, Loss: 1.374
Epoch: 24, Loss: 1.305
Epoch: 25, Loss: 1.324
Epoch: 26, Loss: 1.241
Epoch: 27, Loss: 1.242
Epoch: 28, Loss: 1.195
Epoch: 29, Loss: 1.126
Epoch: 30, Loss: 1.122
Epoch: 31, Loss: 1.088
Epoch: 32, Loss: 1.080
Epoch: 33, Loss: 1.043
Epoch: 34, Loss: 0.996
Epoch: 35, Loss: 0.982
Epoch: 36, Loss: 0.935
Epoch: 37, Loss: 0.916
Epoch: 38, Loss: 0.876
Epoch: 39, Loss: 0.862
Epoch: 40, Loss: 0.856
Epoch: 41, Loss: 0.838
Epoch: 42, Loss: 0.793
Epoch: 43, Loss:

In [20]:
with open('weights.npy', 'wb') as f:
    np.save(f, w_coeff_optimal)
    np.save(f, b_coeff_optimal)
from google.colab import files
files.download('weights.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
############################################################################# splited test set

y_pred_manual = predict(vectorized_X_test.to_numpy(), w_coeff_optimal.T, b_coeff_optimal)
y_pred_manual = np.interp(y_pred_manual, (y_pred_manual.min(), y_pred_manual.max()), (0,5))
lang_mapper = {0:"English", 1:"German", 2:"French", 3:"Persian", 4:"Arabic", 5:"Pashto"}
squarer = lambda t: lang_mapper[int(round(t))]
vfunc = np.vectorize(squarer)
y_pred_manual = vfunc(y_pred_manual)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred_manual))
print(classification_report(y_test,y_pred_manual))
print(accuracy_score(y_test, y_pred_manual))

In [ ]:
############################################################################# wikipedia test set

y_pred_manual = predict(wiki_vectorized_X_test.to_numpy(), w_coeff_optimal.T, b_coeff_optimal)
y_pred_manual = np.interp(y_pred_manual, (y_pred_manual.min(), y_pred_manual.max()), (0,5))
lang_mapper = {0:"English", 1:"German", 2:"French", 3:"Persian", 4:"Arabic", 5:"Pashto"}
squarer = lambda t: lang_mapper[int(round(t))]
vfunc = np.vectorize(squarer)
y_pred_manual = vfunc(y_pred_manual)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(dataset_wikipedia_test.Category,y_pred_manual))
print(classification_report(dataset_wikipedia_test.Category,y_pred_manual))
print(accuracy_score(dataset_wikipedia_test.Category, y_pred_manual))